**CTFM quantitation scripts**

1. The following code was used to derive the photon count for a channel of interest in the window of analysis.

The following packages were used for this analysis:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import lumicks.pylake as lk
from skimage import filters
from scipy.signal import savgol_filter
import pandas as pd

In [ ]:
%matplotlib widget

linear_colormaps = {
"red": LinearSegmentedColormap.from_list("red", colors=[(0, 0, 0), (1, 0, 0)]),
"green": LinearSegmentedColormap.from_list("green", colors=[(0, 0, 0), (0, 1, 0)]),
"blue": LinearSegmentedColormap.from_list("blue", colors=[(0, 0, 0), (0, 0, 1)]),
}

Here onwards, you can load your kymograph in the .h5 format within the portion marked YourFile.h5 below.

In [ ]:
f = lk.File("YourFile.h5")
name, kymo = f.kymos.popitem()

In [ ]:
plt.figure()
kymo.plot("rgb", adjustment=lk.ColorAdjustment([0,0,0], [99,100,100], mode="absolute"),aspect = "auto")
plt.show()

Here, the kymograph is converted to a pixel and frame-index image, to derive any channel-specific photon count information.

In [ ]:
image = kymo.get_image("blue")
plt.figure()
plt.imshow(image,vmax=50, aspect = "auto")
plt.show()

A region is appropriately selected to determine the background photon counts in the kymograph.

In [ ]:
#The first set is along the y-axis (pixels), and the second set is along the x-axis (frame-index))
background = np.mean(image[20:26,460:580]) #These are hypothetical numbers.
background

In [ ]:
image_minus_background = image - background

In [ ]:
#Select region of interest (This will be your window of interest)
selection = image_minus_background[41:45,550:750]

In [ ]:
summed = np.sum(selection, axis=0)
len(summed)

In [ ]:
time_coordinates = np.arange(len(summed))*kymo.line_time_seconds

plt.figure()
plt.plot(time_coordinates,summed)
plt.ylabel("YourLabel")
plt.xlabel("Time (s)")

A Savitzky-Golay filter was applied for purposes of better visualizing trends, in cases where the raw trace is noisy.

In [ ]:
time_coordinates = np.arange(len(summed)) * kymo.line_time_seconds

#Applying the Savitzky-Golay filter to smoothen the data
window_length = 11  
polyorder = 2       

if len(summed) < window_length:
    window_length = len(summed) if len(summed) % 2 == 1 else len(summed) - 1

smoothed = savgol_filter(summed, window_length, polyorder)

#Plot the original and smoothed data here
plt.figure()
plt.plot(time_coordinates, summed, alpha=0.5, label="Raw")
plt.plot(time_coordinates, smoothed, color='r/g/b', label="Smoothed")
plt.ylabel("Blue photon counts")
plt.xlabel("Time (s)")
plt.legend()
plt.show()

#Export variables to a CSV format and save as a CSV file
output_data = np.column_stack((time_coordinates, summed, smoothed))
header = "Time (s),Raw Photon Count,Smoothed Photon Count"
np.savetxt("YourFile.csv", output_data, delimiter=",", header=header, comments='')


2. The following code-chunk was used for applying the Sobel image filter and determining the length of the binding event along the Y-axis.

We end up calling the widget tool because you want to crop out the beads, since they end up providing a region of high contrast change, and can thus confound your image filter.

In [ ]:
widget = kymo.crop_and_calibrate(channel="rgb", aspect="auto", adjustment=lk.ColorAdjustment([0,0,0], [99,100,100], mode="absolute"))

In [ ]:
new_kymo = widget.kymo

plt.figure()
new_kymo.plot("rgb", aspect="auto", adjustment=lk.ColorAdjustment([0,0,0], [99,100,100], mode="absolute"))
plt.show()

In [ ]:
image = new_kymo.get_image("blue")

In [ ]:
plt.figure()
plt.imshow(image,vmax=50, aspect = "auto")
plt.show()

In [ ]:
image = new_kymo.get_image('blue')
edge_filter = filters.sobel(image, axis=[0,0,0])

plt.figure(figsize=(10,6))
plt.imshow(edge_filter, vmax=50)
plt.axis('tight')
plt.tight_layout()
plt.show()

In [ ]:
mock = lk.kymo._kymo_from_array(edge_filter, 'r/g/b', kymo.line_time_seconds, pixel_size_um=1.0)

In [ ]:
plt.close('all')

kw_blue = lk.KymoWidgetGreedy(mock, "r/g/b", pixel_threshold=3, aspect = "auto", min_length=10, window=6, sigma=0.20, vmin = 12, vmax=18, track_width = 5,
    slider_ranges={"sigma": (0.1, 1)}, correct_origin = True)

At this point, you save the tracked regions, and load them when you call the original kymograph next. After loading it, you finally save it again, so the data is calibrated to distance (or knt, if you calibrate the distance to the contour length of the DNA substrate)

In [ ]:
kw_blue_original = lk.KymoWidgetGreedy(new_kymo, "r/g/b",vmin = 25, vmax=45, aspect = "auto", correct_origin=True)

3. The following code chunk was used to determine the positional binding data.

In [ ]:
#Apply background subtraction
background = np.mean(image[7:10, 1000:1300]) #There are random values here
image_minus_background = image - background 

#Set your frame range
start_frame = 45
end_frame = 3560

roi = image_minus_background[1:46, 45:3560] #select your pixels along the y-axis, and frames along the X-axis

frame_window = roi[1:46:, start_frame:end_frame]

average_profile = np.mean(frame_window, axis=1)

# Define pixel positions along tether
pixel_positions = np.arange(average_profile.shape[0])
pixel_size_microns = 0.1  #This value is based on the pixel size of your recordings
positions_um = pixel_positions * pixel_size_microns

# Plotting
plt.figure()
plt.plot(positions_um, average_profile)
plt.xlabel("Position along tether (µm)")
plt.ylabel("Average photon count")
plt.title("Positional Binding Profile (Averaged over Frames {}–{})".format(start_frame, end_frame))
plt.show()

import pandas as pd

# Create a DataFrame
df = pd.DataFrame({
    "Position_um": positions_um,
    "Average_Photon_Count": average_profile
})

# Save to CSV
df.to_csv("YourFile.csv", index=False)